# Fine-tuning XLM-RoBERTa for FR Sentiment Analysis (Text Classification)

This code is available in Hugging Face: 

https://huggingface.co/docs/transformers/tasks/sequence_classification

In [1]:
# Install all necessary libraries

!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
# Login to Hugging Face account

from huggingface_hub import notebook_login

notebook_login()

## Load IMDb dataset

In [3]:
from datasets import load_dataset

imdb = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
print(type(imdb['test']))

<class 'datasets.arrow_dataset.Dataset'>


In [5]:
# Take a look at an example

imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [6]:
imdb["test"][0]['text']

'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as they have

In [7]:
print(type(imdb))

<class 'datasets.dataset_dict.DatasetDict'>


In [8]:
print(len(imdb['test']))

25000


# Load translator

In [9]:
from transformers import pipeline

translator = pipeline("translation_en_to_fr", model="judithrosell/t5-mt-en-fr")

# translator(text)

In [10]:
# Test translator

text = imdb["test"][0]['text']
translation = translator(text, max_length=512)

In [11]:
print(translation)

[{'translation_text': "Je veux m'en tenir à beaucoup. Les films et les télévisions de sci-fi sont généralement sous-financés, sous-appréhendés et mal compris; je m'efforçai de le voir, et c'est à bon sci-fi de télévision, comme Babylon 5 est à Star Trek (l'original); des prothèses silencieuses, des jeux de carton bon marché, des dialogues d'un caractère qui ne correspond pas au fond, et ils ne peuvent pas s'épanouissements d'un caractère à l'autre, et ils sont d'un caractère à l'écouter, et ils ne s'en aperçus à l'origine; et ils ne sont pas à l'origine de Babylones à l'étape de carton à l'origine, des dialogues semblés, des personnages d'un caractère à l'air à l'air à l'origine, et ils ne peuvent pas être en rien à l'uni-fi, et ils sont d'un caractère à l'origine, et ils ne s'épanouissent à l'origine, et ils ne s'étalonnent à l'origine, et ils sont d'un à l'autre, et ils ne s'étalonnent à l'air à l'un à l'autre, et ils ne peuvent pas être entraînés, et ils sont d'un point à l'origine 

# Translate IMDb dataset

In [12]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
# Generate a list of 1500 random ids

import random

# set a seed for reproducibility
random.seed(42)

# select 1500 random ids from 0 to 24999
random_ids = random.sample(range(0, 25000), 1500)

In [14]:
# Generate new imdb TRAIN DatasetDict
new_imdb = {}
new_imdb['train'] = []
#for i in range(len(imdb['train'])):
for i in random_ids[:500]:
    label = imdb['train'][i]['label']
    text = imdb['train'][i]['text']
    sents = nltk.sent_tokenize(text)
    content = ''
    for sent in sents:
        c = translator(sent, max_length=512)
        translated_text = c[0]['translation_text']
        content += translated_text + ' '
    new_imdb['train'].append({'label': label, 'text': content})

Token indices sequence length is longer than the specified maximum sequence length for this model (934 > 512). Running this sequence through the model will result in indexing errors
Your input_length: 934 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


In [15]:
# Generate new imdb TEST DatasetDict
new_imdb['test'] = []
#for i in range(len(imdb['test']))
for i in random_ids[:500]:
    label = imdb['test'][i]['label']
    text = imdb['test'][i]['text']
    sents = nltk.sent_tokenize(text)
    content = ''
    for sent in sents:
        c = translator(sent, max_length=512)
        translated_text = c[0]['translation_text']
        content += translated_text + ' '
    new_imdb['test'].append({'label': label, 'text': content})

In [16]:
# Generate more data for the TRAIN set

for i in random_ids[500:1000]:
    label = imdb['train'][i]['label']
    text = imdb['train'][i]['text']
    sents = nltk.sent_tokenize(text)
    content = ''
    for sent in sents:
        c = translator(sent, max_length=512)
        translated_text = c[0]['translation_text']
        content += translated_text + ' '
    new_imdb['train'].append({'label': label, 'text': content})

In [17]:
# Generate EVEN more data for the TRAIN set

for i in random_ids[1000:1500]:
    label = imdb['train'][i]['label']
    text = imdb['train'][i]['text']
    sents = nltk.sent_tokenize(text)
    content = ''
    for sent in sents:
        c = translator(sent, max_length=512)
        translated_text = c[0]['translation_text']
        content += translated_text + ' '
    new_imdb['train'].append({'label': label, 'text': content})

Your input_length: 1068 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 496 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


In [18]:
new_imdb['train'][0]

{'label': 1,
 'text': "Il s'agit d'un très bon «sequel» plus bon que le premier film d'action en direct 101 Dalmatiens. Il a de bons chiens, de bons acteurs, de bons blagues et tout bon slapstick! – Br – Br – Cruella DeVil, qui a eu une thérapie plutôt importante, est maintenant amant de chiens et très aimable pour eux. Beaucoup, y compris Chloe Simon, propriétaire d'un des chiens que Cruella essayait de tuer, ne croient pas cela. D'autres, comme Kevin Shepherd (propriétaire du 2e refuge de chiens de chance), croient qu'elle a changé. – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – –

In [19]:
new_imdb['train'][:5]

[{'label': 1,
  'text': "Il s'agit d'un très bon «sequel» plus bon que le premier film d'action en direct 101 Dalmatiens. Il a de bons chiens, de bons acteurs, de bons blagues et tout bon slapstick! – Br – Br – Cruella DeVil, qui a eu une thérapie plutôt importante, est maintenant amant de chiens et très aimable pour eux. Beaucoup, y compris Chloe Simon, propriétaire d'un des chiens que Cruella essayait de tuer, ne croient pas cela. D'autres, comme Kevin Shepherd (propriétaire du 2e refuge de chiens de chance), croient qu'elle a changé. – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – –

In [20]:
# Rename new_imdb dataset for TESTING purposes (to see if the code works)

trial_imdb_fr = new_imdb

In [21]:
print(len(trial_imdb_fr['train']))
print(len(trial_imdb_fr['test']))
print(type(trial_imdb_fr['train']))
print(type(trial_imdb_fr['test']))

1500
500
<class 'list'>
<class 'list'>


In [22]:
# Convert lists to datasets.arrow_dataset.Dataset

from datasets import Dataset

# Assuming that each item in the 'train' list is a dictionary
train_data = {"text": [item["text"] for item in trial_imdb_fr['train']],
              "label": [item["label"] for item in trial_imdb_fr['train']]}

train_dataset = Dataset.from_dict(train_data)

In [23]:
# Assuming that each item in the 'test' list is a dictionary
test_data = {"text": [item["text"] for item in trial_imdb_fr['test']],
             "label": [item["label"] for item in trial_imdb_fr['test']]}

test_dataset = Dataset.from_dict(test_data)

In [24]:
# Convert dataset to DatasetDict

from datasets import DatasetDict

new_imdb_dataset = DatasetDict({'train': train_dataset, 'test': test_dataset})

## Preprocess

In [25]:
# Load XLM-RoBERTa tokenizer

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [26]:
# Create a preprocessing function

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [27]:
# Use Datasets map function to apply the preprocessing function over the entire dataset

tokenized_imdb = new_imdb_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [28]:
# Create a batch of examples using DataCollatorWithPadding

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluate

In [29]:
# Import evaluate library

import evaluate

accuracy = evaluate.load("accuracy")

In [30]:
# Create a function that passes your predictions and labels to compute to calculate the accuracy

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Train

In [31]:
# Create a map of the expected ids to their labels with id2label and label2id

id2label = {0: "NEGATIF", 1: "POSITIF"}
label2id = {"NEGATIF": 0, "POSITIF": 1}

In [32]:
# Load XLM-RoBERTa with AutoModelForSequenceClassification

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "xlm-roberta-base", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [33]:
# Define training hyperparameters

training_args = TrainingArguments(
    output_dir="sa_french",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

# Pass the training arguments to Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/judithrosell/sa_french into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.4k/1.04G [00:00<?, ?B/s]

Download file runs/May09_21-25-28_9af657e6726a/events.out.tfevents.1683667603.9af657e6726a.3600.0:  32%|###2  …

Download file training_args.bin: 100%|##########| 3.50k/3.50k [00:00<?, ?B/s]

Clean file runs/May09_21-25-28_9af657e6726a/events.out.tfevents.1683667603.9af657e6726a.3600.0:  23%|##3      …

...

Clean file training_args.bin:  29%|##8       | 1.00k/3.50k [00:00<?, ?B/s]

Clean file runs/May09_21-25-28_9af657e6726a/1683667603.9173882/events.out.tfevents.1683667603.9af657e6726a.360…

Download file tokenizer.json:   0%|          | 1.40k/16.3M [00:00<?, ?B/s]

Clean file tokenizer.json:   0%|          | 1.00k/16.3M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.04G [00:00<?, ?B/s]

In [34]:
# Call train() to finetune the model

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.463406,0.794000
2,No log,0.431486,0.858000
3,No log,0.446116,0.826000
4,No log,0.446220,0.840000
5,No log,0.545508,0.860000


TrainOutput(global_step=470, training_loss=0.3485519084524601, metrics={'train_runtime': 1165.1035, 'train_samples_per_second': 6.437, 'train_steps_per_second': 0.403, 'total_flos': 1967519805320640.0, 'train_loss': 0.3485519084524601, 'epoch': 5.0})

In [35]:
# Save (push) model to HF hub

trainer.push_to_hub()

Upload file runs/May10_01-53-22_bc986e7ae47c/events.out.tfevents.1683683946.bc986e7ae47c.1510.0:   0%|        …

To https://huggingface.co/judithrosell/sa_french
   27ba554..5b7e8c3  main -> main

   27ba554..5b7e8c3  main -> main

To https://huggingface.co/judithrosell/sa_french
   5b7e8c3..fde8991  main -> main

   5b7e8c3..fde8991  main -> main



'https://huggingface.co/judithrosell/sa_french/commit/5b7e8c358a32aa018c32c8aa8addf62ccaf7455f'

## Inference

In [36]:
text = "C'était un chef-d'œuvre. Pas tout à fait fidèle aux livres, mais passionnant du début à la fin. C'est peut-être mon préféré des trois."

In [37]:
# Instantiate a pipeline() for sentiment analysis

from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="judithrosell/sa_french")
classifier(text)

[{'label': 'POSITIF', 'score': 0.8715375065803528}]

In [39]:
# Test a review for clothes

text2 = "Ce jean est super confort, je n'arrête pas de me lisser la jambe car sinon je le sens à peine... et en plus il y a un coté presque amincissant."

classifier(text2)

[{'label': 'POSITIF', 'score': 0.7180951237678528}]